In [56]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
from pathlib import Path
from puzzle import Puzzle, Navigation

In [101]:
puz = Puzzle("inputs.txt")
puz.part_2()

3515583998

In [99]:
puz.lines

[Navigation(raw='[({(<(())[]>[[{[]{<()<>>'),
 Navigation(raw='[(()[<>])]({[<{<<[]>>('),
 Navigation(raw='{([(<{}[<>[]}>{[]{[(<()>'),
 Navigation(raw='(((({<>}<{<{<>}{[]{[]{}'),
 Navigation(raw='[[<[([]))<([[{}[[()]]]'),
 Navigation(raw='[{[{({}]{}}([{[{{{}}([]'),
 Navigation(raw='{<[[]]>}<{[{[{[]{()[[[]'),
 Navigation(raw='[<(<(<(<{}))><([]([]()'),
 Navigation(raw='<{([([[(<>()){}]>(<<{{'),
 Navigation(raw='<{([{{}}[<[[[<>{}]]]>[]]')]

In [51]:
def validate(my_string):
    brackets = ["()", "{}", "[]", "<>"]
    while any(pair in my_string for pair in brackets):
        for br in brackets:
            my_string = my_string.replace(br, "")
    incomplete = set(my_string) - set("({[<") == set()
    invalid = [my_string.find(rt_br) for rt_br in ")}]>"]
    invalid = [x for x in invalid if x != -1]
    if invalid:
        invalid = min(invalid)
    else:
        invalid = None
    return my_string, incomplete, my_string[invalid]


Navigation("{([(<{}[<>[]}>{[]{[(<()>")

[autoreload of puzzle failed: Traceback (most recent call last):
  File "C:\Users\dakas\miniconda3\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\dakas\miniconda3\lib\site-packages\IPython\extensions\autoreload.py", line 394, in superreload
    module = reload(module)
  File "C:\Users\dakas\miniconda3\lib\imp.py", line 314, in reload
    return importlib.reload(module)
  File "C:\Users\dakas\miniconda3\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 855, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "C:\Users\dakas\projects\advent-of-code\2021\day10\puzzle.py", line 11, in <module>
    class Navigation:
  File "C:\Users\dakas\projects\advent-of-code\2021\day10\puzzle.py", line 13, in Navigation
    se

NameError: name 'Navigation' is not defined

In [66]:
my_string = "<"
bool(set(my_string) & set("({[<"))  # == set()

True

In [49]:
validate("[[<[([]))<([[{}[[()]]]")

('[[<[)<([', False, ')')

In [47]:
validate("[({(<(())[]>[[{[]{<()<>>")

('[({([[{{', True, None)

In [39]:
"asdhf".find()

TypeError: find() takes at least 1 argument (0 given)

In [22]:
fname = "tests.txt"
raw = Path(fname).open().readlines()
grid = np.array([list(row.strip()) for row in raw]).astype(int)


low_pts = []

for rownum, row in enumerate(grid):
    for colnum, val in enumerate(row):
        pt = Point(rownum, colnum, grid)
        if pt.is_lowest():
            low_pts.append(pt)
pt

Point(row=4, col=9)

In [57]:
basins = np.where(grid == 9, 0, 1)
basins

array([[1, 1, 0, 0, 0, 1, 1, 1, 1, 1],
       [1, 0, 1, 1, 1, 0, 1, 0, 1, 1],
       [0, 1, 1, 1, 1, 1, 0, 1, 0, 1],
       [1, 1, 1, 1, 1, 0, 1, 1, 1, 0],
       [0, 1, 0, 0, 0, 1, 1, 1, 1, 1]])

In [60]:
from scipy.ndimage import measurements

lw, num = measurements.label(basins)
area = measurements.sum(basins, lw, index=np.arange(lw.max() + 1))
area

array([ 0.,  3.,  9., 14.,  9.])

## Black format my final answer

In [102]:
from pathlib import Path
from dataclasses import dataclass, field
from statistics import median


T1_ANS = 26397
T2_ANS = 288957


@dataclass
class Navigation:
    raw: str
    incomplete: bool = field(default=None, repr=False)
    invalid: str = field(default=None, repr=False)

    def __post_init__(self):
        self.validate()
        return

    def validate(self):
        """Adapted from approach #3:
        https://www.geeksforgeeks.org/check-for-balanced-parentheses-in-python/
        """
        my_string = self.raw
        brackets = ["()", "{}", "[]", "<>"]
        while any(pair in my_string for pair in brackets):
            for br in brackets:
                my_string = my_string.replace(br, "")
        self.incomplete = set(my_string).issubset(set("({[<"))
        if self.incomplete:
            self.needs_completing = my_string
        invalid_idx = [my_string.find(rt_br) for rt_br in ")}]>"]
        invalid_idx = [x for x in invalid_idx if x != -1]
        if invalid_idx:
            self.invalid = my_string[min(invalid_idx)]
        return self.incomplete, self.invalid

    def complete(self):
        """invalid takes precedence over incomplete, so if it
        is both, this code wil NOT complete an invalid line.
        """
        if not self.incomplete:
            return ""
        closer = {"(": ")", "{": "}", "[": "]", "<": ">"}
        return "".join(closer[b] for b in reversed(self.needs_completing))


@dataclass
class Puzzle:
    fname: str
    lines: list = None

    def __post_init__(self):
        raw = Path(self.fname).open().readlines()
        self.lines = [Navigation(line.strip()) for line in raw]

    def part_1(self):
        todo = [l for l in self.lines if l.invalid and not l.incomplete]
        scoring = {")": 3, "]": 57, "}": 1197, ">": 25137}
        return sum(scoring[line.invalid] for line in todo)

    def part_2(self):
        todo = [l for l in self.lines if l.incomplete and not l.invalid]
        scoring = {")": 1, "]": 2, "}": 3, ">": 4}

        scores = []
        for l in todo:
            r_brackets = l.complete()
            score = 0
            for br in r_brackets:
                score *= 5
                score += scoring[br]
            scores.append(score)

        return median(scores)


def run_tests(p1_ans=T1_ANS, p2_ans=T2_ANS, fname="tests.txt"):
    puz = Puzzle(fname)
    t1 = puz.part_1()
    assert t1 == p1_ans, f"Test 1 failed. Got {t1} instead of {p1_ans}"

    if p2_ans is not None:
        t2 = puz.part_2()
        assert t2 == p2_ans, f"Test 2 failed. Got {t2} instead of {p2_ans}"

    print("All tests passed.")
    return


if __name__ == "__main__":
    run_tests()

    puz = Puzzle("inputs.txt")

    p1 = puz.part_1()
    print("Part 1:", p1)

    if T2_ANS is not None:
        p2 = puz.part_2()
        print("Part 2:", p2)

All tests passed.
Part 1: 319329
Part 2: 3515583998


In [5]:
import numpy as np
from scipy import ndimage

# floor = np.array(
#     [
#         [2, 1, 9, 9, 9, 4, 3, 2, 1, 0],
#         [3, 9, 8, 7, 8, 9, 4, 9, 2, 1],
#         [9, 8, 5, 6, 7, 8, 9, 8, 9, 2],
#         [8, 7, 6, 7, 8, 9, 6, 7, 8, 9],
#         [9, 8, 9, 9, 9, 6, 5, 6, 7, 8],
#     ]
# )

floor = puz.grid

mask = np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]])

window_minima = ndimage.minimum_filter(floor, footprint=mask, mode="constant", cval=9)
minima = floor[floor == window_minima]
sum(minima + 1)

1654

In [6]:
np.where(floor == window_minima, 1, 0)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])